## AndLink and EvaluationLink examples

In [ ]:
import common
from opencog.utilities import initialize_opencog
from opencog.atomspace import AtomSpace, types
from opencog.type_constructors import *
from opencog.bindlink import execute_atom

create atomspace and set it as default

In [ ]:
atomspace = AtomSpace()
initialize_opencog(atomspace)

### knowledge base

Color and size are attributes.  
InheritanceLink is similiar to verb "is" like it is used in natural languages.

In [ ]:
InheritanceLink(ConceptNode("size"), ConceptNode("attribute"))
InheritanceLink(ConceptNode("color"), ConceptNode("attribute"))

Different types of color and size  
PredicateNode is used to define relations between different objects.

In [ ]:
InheritanceLink(PredicateNode("small"), ConceptNode("size"))
InheritanceLink(PredicateNode("big"), ConceptNode("size"))
InheritanceLink(PredicateNode("green"), ConceptNode("color"))
InheritanceLink(PredicateNode("red"), ConceptNode("color"))
InheritanceLink(PredicateNode("black"), ConceptNode("color"))

animals which have some attributes defined using EvaluationLink  
EvaluationLink defines predicate for particular objects.

In [ ]:
def assign_property(concept, relation, tv):
    EvaluationLink(relation,
                   concept).tv = TruthValue(*tv)

assign_property(ConceptNode("frog"), PredicateNode("small"), (0.8, 0.6))
assign_property(ConceptNode("frog"), PredicateNode("green"), (0.7, 0.91))
assign_property(ConceptNode("tiger"), PredicateNode("big"), (0.9, 0.92))
assign_property(ConceptNode("tiger"), PredicateNode("red"), (0.6, 0.83))
assign_property(ConceptNode("sparrow"), PredicateNode("red"), (0.1, 0.64))
assign_property(ConceptNode("sparrow"), PredicateNode("small"), (0.8, 0.77))
assign_property(ConceptNode("zebra"), PredicateNode("black"), (0.5, 0.9))
assign_property(ConceptNode("zebra"), PredicateNode("small"), (0.38, 0.89))

Define that all animals are actually animals

In [ ]:
InheritanceLink(ConceptNode("frog"), ConceptNode("animal"))
InheritanceLink(ConceptNode("frog"), ConceptNode("animal"))
InheritanceLink(ConceptNode("tiger"), ConceptNode("animal"))
InheritanceLink(ConceptNode("tiger"), ConceptNode("animal"))
InheritanceLink(ConceptNode("sparrow"), ConceptNode("animal"))
InheritanceLink(ConceptNode("sparrow"), ConceptNode("animal"))
InheritanceLink(ConceptNode("zebra"), ConceptNode("animal"))
InheritanceLink(ConceptNode("zebra"), ConceptNode("animal"))

**Query example**  
let's select all animals and attributes, filtering by some threshold


first define function and EvaluationLink

In [ ]:
threshold = 0.6

def apply_threshold(atom):
    if atom.tv.mean < threshold:
        return TruthValue(0, 1)
    return TruthValue(1, 1)

Constrain Variable("X") is animal

In [ ]:
is_animal = InheritanceLink(VariableNode("X"), ConceptNode("animal"))

EvaluationLink will accept link    X <- "animal" 
and return flag if it "matches"

In [ ]:
predicate = EvaluationLink(VariableNode("$B"),
                           VariableNode("X"))

eval_threshold = EvaluationLink(
                    GroundedPredicateNode("py: apply_threshold"),
                    ListLink(predicate))

first define pattern that will be grounded during the search

In [ ]:
and_link = AndLink(is_animal,
                   predicate,
                   eval_threshold
                  )

### now build the BindLink
BindLink is constructed using pattern to ground and pattern to return.  
pattern to return if it contains variables,  
should be part of pattern to ground

In [ ]:
bindlink_filter = BindLink(and_link,
                           predicate)

execute query

In [ ]:
print(execute_atom(atomspace, bindlink_filter))

## Exercise: use atom.tv.confidence

in threshold function